# Gestion de archivos

En esta clase vamos a ver cómo podemos introducir archivos dentro de nuestro asistente. Los archivos permiten al asistente aprender nuevas informaciones. Además también podemos manipular dichos archivos a través del asistente.

Veamos un ejemplo de cómo podríamos utilizar esta funcionalidad. Empezamos comos siempre instalando el módulo de OpenAI e introduciendo nuestra clave API:

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [3]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-proj-JgnyPrVpPr4xoz1FrPxiT3BlbkFJcIB6dz6ptu7gA5UifDsQ"
openai.api_key = os.environ["OPENAI_API_KEY"]
client = openai.OpenAI()

## Creación del asistente

Comenzamos creando el asistente indicando qué usaremos archivos mediante el parámetro `tools`:

In [4]:
instrucciones_asistente = """
Eres un profesor de Python para principiantes. Los usuarios te preguntarán sobre
temas relacionados con programación y tú debes ayudarlos. Para ello les explicarás
brevemente la teoría detrás de los conceptos implicados, la sintaxis general para
resolver la duda y por último darás un ejemplo de código comentando en cada línea
de código lo que estás haciendo.
"""

In [5]:
asistente_archivos = client.beta.assistants.create(
  name="Profe de Python varchivos",
  description="Versión mejorada incluyendo gestión de archivos",
  instructions=instrucciones_asistente,
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

## Creación del espacio de almacenamiento

A continuación vamos a almacenar los archivos para lo que empezamos creando un `vector_store`:

In [6]:
vector_store = client.beta.vector_stores.create(name="Archivos Python Profe v2")

In [8]:
vector_store.id

'vs_7PqchTRWmLuvIxWbsv1Jn7fU'

Creemos un par de archivos de ejemplo para trabajar con ellos:

In [7]:
ejemplo_lista = ["plátano", "naranja", "manzana", "cereza", "pera", "melocotón", "sandía"]
nombre_archivo = "mi_lista.txt"
with open(nombre_archivo, "w") as archivo:
    for elemento in ejemplo_lista:
        archivo.write(elemento + "\n")

A continuación vamos a cargar estos dos archivos de prueba en nuestro espacio de almacenamiento. Podemos hacerlo de forma manual mediante la web de OpenAI como una subida normal de archivos o podemos hacerlo de manera programática:

In [9]:
direccion_archivo = ["mi_lista.txt", "muestra_pdf.pdf"]
informacion_archivos = [open(ruta, "rb") for ruta in direccion_archivo]


carga_de_archivos = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=informacion_archivos
)


De nuevo tenemos tanto la opción de la interfaz como la programática para comprobar que los archivos se han subido correctamente:

In [10]:
print(carga_de_archivos.status)
print(carga_de_archivos.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


## Conexión del asistente y el almacenamiento

A continuación vamos a indicar al asistente que puede usar los archivos como recursos:

In [11]:
asistente_archivos = client.beta.assistants.update(
  assistant_id=asistente_archivos.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

Ya hemos cargado la información. Veamos que funciona de manera correcta:

In [12]:
hilo = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "¿Cómo me llamo y cuántos años tengo? Dime además las frutas del archivo en orden alfabético"
    }
  ]
)

In [13]:
ejecucion = client.beta.threads.runs.create(
  thread_id=hilo.id,
  assistant_id=asistente_archivos.id
)

In [14]:
def recupera_conversacion_de_hilo(hilo):
  conversacion = {}
  mensajes_hilo = client.beta.threads.messages.list(hilo)
  longitud_hilo = len(mensajes_hilo.data)
  for indice in range(0, longitud_hilo):
    informacion_mensaje = mensajes_hilo.data[longitud_hilo-1-indice]
    conversacion[indice] = (informacion_mensaje.role, informacion_mensaje.content[0].text.value)

  return conversacion

conversacion_1 = recupera_conversacion_de_hilo(hilo.id)

In [15]:
conversacion_1

{0: ('user',
  '¿Cómo me llamo y cuántos años tengo? Dime además las frutas del archivo en orden alfabético'),
 1: ('assistant',
  'Te llamas Arturo Sánchez Palacio, y tienes 28 años【4:0†muestra_pdf.pdf】.\n\nLas frutas en el archivo, ordenadas alfabéticamente, son:\n\n1. Cereza\n2. Manzana\n3. Melocotón\n4. Naranja\n5. Pera\n6. Plátano\n7. Sandía【4:1†mi_lista.txt】.')}

Como podemos observar cuando lo hacemos de manera programática se generan caracteres extraños. Veamos cómo podemos arreglar esto. Cargamos los mensajes contenidos hasta ahora en el hilo:

In [16]:
mensajes = list(client.beta.threads.messages.list(thread_id=hilo.id, run_id=ejecucion.id))
mensajes

[Message(id='msg_SvDsxJJTaDrxjZoptgq2KjKQ', assistant_id='asst_C3WdI6YyhvpKsXUGIqWIY0R5', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=71, file_citation=FileCitation(file_id='file-wXRDADsLprGpDAeNNZRz1ZiO', quote=None), start_index=50, text='【4:0†muestra_pdf.pdf】', type='file_citation'), FileCitationAnnotation(end_index=224, file_citation=FileCitation(file_id='file-gunNrNQ6O0ws0vmbmh8O7yBp', quote=None), start_index=206, text='【4:1†mi_lista.txt】', type='file_citation')], value='Te llamas Arturo Sánchez Palacio, y tienes 28 años【4:0†muestra_pdf.pdf】.\n\nLas frutas en el archivo, ordenadas alfabéticamente, son:\n\n1. Cereza\n2. Manzana\n3. Melocotón\n4. Naranja\n5. Pera\n6. Plátano\n7. Sandía【4:1†mi_lista.txt】.'), type='text')], created_at=1718211402, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_FBwgFfmjqhEDvKnlnYR65lzG', status=None, thread_id='thre

Extraemos el contenido de los mensajes y las anotaciones:

In [18]:
contenidos_mensajes = mensajes[0].content[0].text
contenidos_mensajes.value

'Te llamas Arturo Sánchez Palacio, y tienes 28 años【4:0†muestra_pdf.pdf】.\n\nLas frutas en el archivo, ordenadas alfabéticamente, son:\n\n1. Cereza\n2. Manzana\n3. Melocotón\n4. Naranja\n5. Pera\n6. Plátano\n7. Sandía【4:1†mi_lista.txt】.'

In [19]:
anotaciones = contenidos_mensajes.annotations
anotaciones

[FileCitationAnnotation(end_index=71, file_citation=FileCitation(file_id='file-wXRDADsLprGpDAeNNZRz1ZiO', quote=None), start_index=50, text='【4:0†muestra_pdf.pdf】', type='file_citation'),
 FileCitationAnnotation(end_index=224, file_citation=FileCitation(file_id='file-gunNrNQ6O0ws0vmbmh8O7yBp', quote=None), start_index=206, text='【4:1†mi_lista.txt】', type='file_citation')]

Para extraer las referencias vamos recorriendo las anotiaciones y las sustituimos por un índice que usaremos como ancla de la referencia.

Después añadimos al final del mensaje las referencias junto a su ancla

In [20]:
referencias = []
for indice, anotacion in enumerate(anotaciones):
  contenidos_mensajes.value = contenidos_mensajes.value.replace(anotacion.text, f"[{indice}]")
  if file_citation := getattr(anotacion, "file_citation", None):
    archivo_citado = client.files.retrieve(file_citation.file_id)
    referencias.append(f"[{indice}] {archivo_citado.filename}")


In [21]:
print(contenidos_mensajes.value)
print("\n".join(referencias))

Te llamas Arturo Sánchez Palacio, y tienes 28 años[0].

Las frutas en el archivo, ordenadas alfabéticamente, son:

1. Cereza
2. Manzana
3. Melocotón
4. Naranja
5. Pera
6. Plátano
7. Sandía[1].
[0] muestra_pdf.pdf
[1] mi_lista.txt


## Cierre

En esta clase hemos visto cómo podemos cargar y utilizar archivos dentro de nuestro asistente virtual.

Te recomiendo que pruebes con tus propios archivos y compruebas cómo además es posible subir muchos más archivos.